In [1]:
import cv2
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from yolov4.tf import YOLOv4
cv2.__version__

Call tf.config.experimental.set_memory_growth(GPU0, True)


'4.4.0'

In [2]:
yolo = YOLOv4()

yolo.config.parse_names("coco.names")
yolo.config.parse_cfg("yolov4-tiny.cfg")

yolo.make_model()
yolo.load_weights("logs/yolov4-tiny-22000-step.weights", weights_type='yolo')
yolo.summary(summary_type="yolo", gpu=1)
yolo.summary()

yolo.inference(media_path="dog.jpg")

# yolo.inference(media_path="test2.mp4", is_image=False, cv_frame_size=(416,416))

# yolo.inference(
#     "/dev/video0",
#     is_image=False,
#     cv_apiPreference=cv2.CAP_V4L2,
#     cv_frame_size=(418, 418),
#     cv_fourcc="YUYV",
# )

batch: 32
index layer No. filters  size/strd(dil)   input_shape         output_shape    1e9 flops
   0  convo_0      32      3 x 3 / 2      416 x 416 x   3 ->  208 x 208 x  32   0.075
   1  convo_1      64      3 x 3 / 2      208 x 208 x  32 ->  104 x 104 x  64   0.399
   2  convo_2      64      3 x 3 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.797
   3  route_0       2,                                    ->  104 x 104 x  32
   4  convo_3      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   5  convo_4      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   6  route_1       5,  4,                                ->  104 x 104 x  64
   7  convo_5      64      1 x 1 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.089
   8  route_2       2,  7,                                ->  104 x 104 x 128
   9  maxpo_0              2 x 2 / 2      104 x 104 x 128 ->   52 x  52 x 128   0.001
  10  convo_6     128      3 x 3 / 1       52 x  52 x 128 ->   52 

In [4]:
# Yolo 로드
net = cv2.dnn.readNet("logs/yolov4-tiny-22000-step.weights", "yolov4-tiny.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [5]:
# 이미지 가져오기
img = cv2.imread("D:/yolo_object_detection/room_ser.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [6]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [7]:
# 정보를 화면에 표시
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.5)

In [9]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
## Train
from tensorflow.keras import callbacks

from yolov4.tf import YOLOv4, YOLODataset, SaveWeightsCallback

yolo = YOLOv4()

yolo.config.parse_names("coco.names")
yolo.config.parse_cfg("yolov4-tiny.cfg")

yolo.make_model()
yolo.load_weights(
    "yolov4-tiny.weights",
    weights_type="yolo",
)
yolo.summary(summary_type="yolo")

for i in range(29):
    yolo.model.get_layer(index=i).trainable = False

yolo.summary()

train_dataset = YOLODataset(
    config=yolo.config,
    dataset_list="data/dataset/train_test.txt",
    training=True,
)

val_dataset = YOLODataset(
    config=yolo.config,
    dataset_list="data/dataset/val_test.txt",
    # image_path_prefix="/content/val2017",
    training=False,
)

yolo.compile()

_callbacks = [
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        log_dir="logs/",
        update_freq=200,
        histogram_freq=1,
    ),
    SaveWeightsCallback(
        yolo=yolo,
        dir_path="logs/",
        weights_type="yolo",
        step_per_save=2000,
    ),
]

yolo.fit(
    train_dataset,
    callbacks=_callbacks,
    validation_data=val_dataset,
    verbose=3  # 3: print step info
)

batch: 32
index layer No. filters  size/strd(dil)   input_shape         output_shape    1e9 flops
   0  convo_0      32      3 x 3 / 2      416 x 416 x   3 ->  208 x 208 x  32   0.075
   1  convo_1      64      3 x 3 / 2      208 x 208 x  32 ->  104 x 104 x  64   0.399
   2  convo_2      64      3 x 3 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.797
   3  route_0       2,                                    ->  104 x 104 x  32
   4  convo_3      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   5  convo_4      32      3 x 3 / 1      104 x 104 x  32 ->  104 x 104 x  32   0.199
   6  route_1       5,  4,                                ->  104 x 104 x  64
   7  convo_5      64      1 x 1 / 1      104 x 104 x  64 ->  104 x 104 x  64   0.089
   8  route_2       2,  7,                                ->  104 x 104 x 128
   9  maxpo_0              2 x 2 / 2      104 x 104 x 128 ->   52 x  52 x 128   0.001
  10  convo_6     128      3 x 3 / 1       52 x  52 x 128 ->   52 

In [32]:
# tf_set에 맞춰 파일 세팅
from glob import glob
train_set = []
for i in glob('E:/ai_set/*.txt'):

    try:
        with open(i, encoding='utf-8') as f:
            tmp_file = f.read().split()
            content = i + ' ' + tmp_file[0] + ' ' + tmp_file[1] + ' ' + tmp_file[2] + ' ' + tmp_file[3] + ' ' + tmp_file[4] # Yolov4 data set
            with open('data/dataset/re_set.txt', 'a', encoding='utf-8') as ff:
                ff.write(content+'\n')
                ff.close()
            f.close()
    except Exception as ex:
        print(ex)
        print(i)
        pass

In [40]:
from sklearn.model_selection import train_test_split

X = []
with open('data/dataset/re_set.txt', encoding='utf-8') as f:
    for i in f:
        X.append((i.replace('.txt','.jpg')).split('\n')[0])
# print(X[0].split()[1])
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test = train_test_split(X, test_size=1 - train_ratio, shuffle=True, random_state = 42)
x_val, x_test = train_test_split(x_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=True, random_state = 42)

for i in x_train:
    with open('D:/yolov4_TF/data/dataset/re_train.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()
for i in x_val:
    with open('D:/yolov4_TF/data/dataset/re_val.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()
for i in x_test:
    with open('D:/yolov4_TF/data/dataset/re_test.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()


In [22]:
from glob import glob
import os
for i,j in enumerate(glob('C:/Users/HF/Desktop/mingsu/object_detection/Tensorflow/workspace/images/test/*.xml')):
    # if i ==13 :
    #     # print(j)
    #     break
    os.rename(j, j.replace('Steve ',''))
    # print(j)